# Notebook: Train Model

## Packages

In [1]:
from load_dataset_folds import load_dataset_folds
from transformers import set_seed
from ACD import train_ACD_model
from E2E import train_E2E_model
from OTE import train_OTE_model
from ACSA import train_ACSA_model
from TASD import train_TASD_model
import pandas as pd
import numpy as np
import constants
import warnings
import shutil
import torch
import json
import sys
import os

## Parameters

In [2]:
LLM_NAME = "Llama13B"
N_REAL = 500
N_SYNTH = 0
TARGET = "target_aspect_sentiment_detection" # "aspect_category", "aspect_category_sentiment", "end_2_end_absa" ,"target_aspect_sentiment_detection"
LLM_SAMPLING = "random"

## Settings

In [3]:
# Set seeds
torch.device(constants.DEVICE)
torch.manual_seed(constants.RANDOM_SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(constants.RANDOM_SEED)
set_seed(constants.RANDOM_SEED)

if torch.cuda.is_available():
    torch.cuda.manual_seed_all(constants.RANDOM_SEED)

# Ignore warnings
warnings.filterwarnings("ignore", category=FutureWarning,
                        module="transformers.optimization")

# Disable Pycache
sys.dont_write_bytecode = True

## Code

### Create Folders for Results

In [ ]:
folders = ['split_results', 'results_csv', 'results_json']
for folder in folders:
    if not os.path.exists(folder):
        os.makedirs(folder)

### Load Dataset

In [4]:
train_dataset, test_dataset = load_dataset_folds(LLM_NAME, N_REAL, N_SYNTH, LLM_SAMPLING)

500 5
500 5


### Load Model

In [5]:
if TARGET == "aspect_category":
    results = train_ACD_model(
        LLM_NAME, N_REAL, N_SYNTH, TARGET, LLM_SAMPLING, train_dataset, test_dataset)

In [6]:
if TARGET == "aspect_category_sentiment":
    results = train_ACSA_model(LLM_NAME, N_REAL, N_SYNTH, TARGET, LLM_SAMPLING, train_dataset, test_dataset)

In [8]:
if TARGET == "end_2_end_absa":
    results = train_E2E_model(LLM_NAME, N_REAL, N_SYNTH, TARGET, LLM_SAMPLING, train_dataset, test_dataset)

In [9]:
if TARGET == "aspect_term":
    results = train_OTE_model(
        LLM_NAME, N_REAL, N_SYNTH, TARGET, LLM_SAMPLING, train_dataset, test_dataset)

In [10]:
if TARGET == "target_aspect_sentiment_detection":
    results = train_TASD_model(
        LLM_NAME, N_REAL, N_SYNTH, TARGET, LLM_SAMPLING, train_dataset, test_dataset)

/opt/homebrew/Caskroom/miniforge/base/envs/tensorflow_m1/lib/python3.11/site-packages/transformers/models/t5/tokenization_t5.py:238: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so n

AssertionError: Torch not compiled with CUDA enabled

### Save Results

In [ ]:
with open(f'results_json/results_{LLM_NAME}_real{N_REAL}_synth{N_SYNTH}_{TARGET}_{LLM_SAMPLING}.json', 'w') as json_file:
    json.dump(results, json_file)

df = pd.DataFrame([results])
df.to_csv(f'results_csv/results_{LLM_NAME}_real{N_REAL}_synth{N_SYNTH}_{TARGET}_{LLM_SAMPLING}.csv', index=False)

In [ ]:
results

### Remove useless folders

In [ ]:
try:
    shutil.rmtree("outputs")
except:
    pass